# SPARCL + Jdaviz
This is very promising. I am thinking that it would be useful to produce a full workflow with a list of spectra (instead of just one) and figure out the best way to call Specviz one spectrum at a time.

The installation of `sperclclient` + `jdaviz` has a version conflict in the package `requests`. This notebook works if `jdaviz` is installed first and the environment takes the requirement of `request` from `sparclclient` (installed second).  
`pip install jdaviz`  
`pip install sparclclient`

In [ ]:
# SPARCL import
from sparcl.client import SparclClient
# Jdaviz import
from jdaviz import Specviz
# Specutils import
from specutils import Spectrum1D
# Astropy import
import astropy.units as u
from astropy.nddata import InverseVariance 

## Set up the SPARCL client

In [ ]:
client = SparclClient()
client.all_datasets

In [ ]:
client.get_all_fields(dataset_list=['SDSS-DR16'])

## Run a search

### Notes
It took me a few attemps to understand the keywords I can put in `out`.  
I later realized that I had to wait for the retrieval to get `flux` and `wavelength` (unless I am doing something wrong).

In [ ]:
out = ['id', 'ra', 'dec', 'spectype', 'specid', 'redshift', 'specprimary']
cons = {'redshift': [0.5, 0.55],
        'data_release': ['SDSS-DR16']}

In [ ]:
found = client.find(outfields=out,
                    constraints=cons,
                    limit=5)

In [ ]:
found.records[0]

In [ ]:
ids = found.ids
ids

## Retrieve the records

### Notes
The retrieval is pretty fast. I am assuming the `model` goes on the same `wavelength` of the observed spectrum.

In [ ]:
result = client.retrieve(uuid_list=ids,
                         include=['specid', 
                                  'redshift', 
                                  'flux', 
                                  'ivar', 
                                  'wavelength', 
                                  'mask', 
                                  'model'])
result.info

In [ ]:
onespec = result.records[0]
onespec

## Create a Spectrum1D object

In [ ]:
specunit = u.Unit('erg cm-2 s-1 AA-1')
spec1d = Spectrum1D(spectral_axis = onespec['wavelength']*u.AA,
                    flux = onespec['flux']*specunit,
                    uncertainty = InverseVariance(onespec['ivar']),
                    mask = onespec['mask'],
                    redshift = onespec['redshift']
                   )
spec1dmodel = Spectrum1D(spectral_axis = onespec['wavelength']*u.AA,
                         flux = onespec['model']*specunit)
spec1d

## Open in Specviz

### Notes
We will probably want calls to the API to do some basic stuff like:
- change plot options
- smooth the spectrum
- plot some lines
- load the redshift
- do some line analysis too maybe?

In [ ]:
specviz = Specviz()
specviz.load_data(spec1d, data_label='observation')
specviz.load_data(spec1dmodel, data_label='model')
specviz.show()